In [31]:
from abc import ABC, abstractmethod
import queue
import threading

# Command Interface
class Command(ABC):
    @abstractmethod
    def execute(self):
        pass

# Concrete Command 1: Financial Calculation
class FinancialCalculationCommand(Command):
    def __init__(self, data):
        self.data = data

    def execute(self):
        # Simulate a financial calculation
        result = sum(self.data) / len(self.data)
        print(f"Financial calculation result: {result}")

# Concrete Command 2: Network Fetch
class NetworkFetchCommand(Command):
    def __init__(self, url):
        self.url = url

    def execute(self):
        # Simulate network fetching (dummy implementation)
        print(f"Fetching data from {self.url}")
        # Normally, you would use requests or similar here

# Job Queue
class JobQueue:
    def __init__(self, max_threads):
        self.queue = queue.Queue()
        self.max_threads = max_threads
        self.threads = []

        for _ in range(max_threads):
            worker = threading.Thread(target=self.worker)
            worker.daemon = True
            worker.start()
            self.threads.append(worker)

    def worker(self):
        while True:
            command = self.queue.get()
            if command is None:  # Stop signal
                break
            command.execute()
            self.queue.task_done()

    def add_job(self, command):
        self.queue.put(command)

    def wait_completion(self):
        self.queue.join()

    def stop_workers(self):
        for _ in range(self.max_threads):
            self.queue.put(None)  # Stop all threads
        for t in self.threads:
            t.join()

# Example Usage
if __name__ == "__main__":
    job_queue = JobQueue(max_threads=3)

    # Adding different types of commands to the queue
    job_queue.add_job(FinancialCalculationCommand([100, 200, 300]))
    job_queue.add_job(NetworkFetchCommand("https://example.com"))
    job_queue.add_job(FinancialCalculationCommand([400, 500, 600]))
    job_queue.add_job(NetworkFetchCommand("https://another-example.com"))

    # Wait for all tasks to complete
    job_queue.wait_completion()
    job_queue.stop_workers()

    print("All tasks are completed.")


Financial calculation result: 200.0Fetching data from https://example.com
Financial calculation result: 500.0
Fetching data from https://another-example.com

All tasks are completed.
